In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import geoplot as gplt
import datetime
import pyproj
from shapely.geometry import Polygon, MultiPolygon,Point, MultiPoint
from shapely.ops import nearest_points

## Fire box

In [2]:
firebox = gpd.read_file('data/firebox_all.geojson')


In [3]:
firetable = pd.read_csv('data/Fire_Incident_Dispatch_Data.csv')
firemask = firetable.INCIDENT_CLASSIFICATION_GROUP.apply(lambda x: x in ['Structural Fires'])
firetable = firetable[firemask]
firetable.reset_index(inplace=True)
firetable.head()

,index,STARFIRE_INCIDENT_ID,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,ALARM_BOX_NUMBER,ALARM_BOX_LOCATION,INCIDENT_BOROUGH,ZIPCODE,POLICEPRECINCT,CITYCOUNCILDISTRICT,...,FIRST_ACTIVATION_DATETIME,FIRST_ON_SCENE_DATETIME,INCIDENT_CLOSE_DATETIME,VALID_DISPATCH_RSPNS_TIME_INDC,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
0,58,1.300114e+15,01/01/2013 12:34:10 AM,QUEENS,1380,OCEAN PROMENADE & BEACH 117 ST,QUEENS,11694.0,100.0,32.0,...,01/01/2013 12:34:54 AM,01/01/2013 12:37:52 AM,01/01/2013 12:49:43 AM,N,Y,222,197,3,3,1
1,65,1.300112e+15,01/01/2013 12:38:51 AM,QUEENS,1201,BEACH CHANNEL DR & B41 ST,QUEENS,11691.0,101.0,31.0,...,01/01/2013 12:39:39 AM,01/01/2013 12:42:40 AM,01/01/2013 12:57:58 AM,N,Y,229,220,3,3,1
2,105,1.300126e+15,01/01/2013 12:58:10 AM,BROOKLYN,2595,11 AVE & 60 ST,BROOKLYN,11219.0,66.0,38.0,...,01/01/2013 12:58:34 AM,01/01/2013 01:00:50 AM,01/01/2013 04:15:20 AM,N,Y,160,151,5,4,7
3,143,1.300125e+15,01/01/2013 01:23:26 AM,BRONX,2538,CLAY AVE & 166 ST,BRONX,10456.0,44.0,16.0,...,01/01/2013 01:24:21 AM,01/01/2013 01:27:05 AM,01/01/2013 01:37:33 AM,N,Y,219,186,4,2,1
4,237,1.300131e+15,01/01/2013 02:16:55 AM,BRONX,3143,ARTHUR AVE & 180 ST,BRONX,10457.0,48.0,15.0,...,01/01/2013 02:17:34 AM,01/01/2013 02:21:06 AM,01/01/2013 02:36:23 AM,N,Y,251,232,3,2,1


In [4]:
firetable.INCIDENT_DATETIME = firetable.INCIDENT_DATETIME.apply(lambda x:\
                datetime.datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))


In [5]:
firetable13 = firetable[firetable.INCIDENT_DATETIME<datetime.date(2014,1,1)]
firetable14 = firetable[(firetable.INCIDENT_DATETIME<datetime.date(2015,1,1)) &
                  (firetable.INCIDENT_DATETIME>=datetime.date(2014,1,1))]
firetable15 = firetable[(firetable.INCIDENT_DATETIME<datetime.date(2016,1,1)) &
                  (firetable.INCIDENT_DATETIME>=datetime.date(2015,1,1))]

In [6]:
boro13 = firetable13.ALARM_BOX_BOROUGH
num13 = firetable13.ALARM_BOX_NUMBER
level13 = firetable13.HIGHEST_ALARM_LEVEL
boro14 = firetable14.ALARM_BOX_BOROUGH
num14 = firetable14.ALARM_BOX_NUMBER
level14 = firetable14.HIGHEST_ALARM_LEVEL
boro15 = firetable15.ALARM_BOX_BOROUGH
num15 = firetable15.ALARM_BOX_NUMBER
level15 = firetable15.HIGHEST_ALARM_LEVEL

In [7]:
count13 = firebox.apply(lambda x:\
                        np.sum((boro13==x['ALARM_BOX_BOROUGH'])&
                               (num13==x['ALARM_BOX_NUMBER'])),
                           axis='columns')

count14 = firebox.apply(lambda x:\
                        np.sum((boro14==x['ALARM_BOX_BOROUGH'])&
                               (num14==x['ALARM_BOX_NUMBER'])),
                           axis='columns')

count15 = firebox.apply(lambda x:\
                        np.sum((boro15==x['ALARM_BOX_BOROUGH'])&
                               (num15==x['ALARM_BOX_NUMBER'])),
                           axis='columns')

In [8]:
firebox['count_2013'] = count13
firebox['count_2014'] = count14
firebox['count_2015'] = count15

## Maintenance Violations

In [ ]:
vol = pd.read_csv('data/Housing_Maintenance_Code_Violations.csv')

In [ ]:
vol_feature=vol[['BuildingID','BoroID', 'Block', 'Lot', 'Class', 'NOVDescription','ViolationStatus', 'CurrentStatus', 'Latitude',
       'Longitude','BIN','BBL', 'InspectionDate', 'CurrentStatusDate']]

In [ ]:
new_bbl = vol_feature.apply(lambda x: int('{0}{1:05d}{2:04d}'.format(x.BoroID, 
                                                                 x.Block, x.Lot)), axis=1)
vol_feature['BBL_new'] = new_bbl

In [ ]:
vol_feature.drop(columns=['Latitude', 'Longitude', 'BIN', 'BBL'], inplace=True)
vol_feature.dropna(inplace=True)

In [ ]:
vol_feature['InspectionDate'] = vol_feature.apply(lambda x:\
                                                  datetime.datetime.strptime(x.InspectionDate, '%m/%d/%Y'), axis=1)
vol_feature['CurrentStatusDate'] = vol_feature.apply(lambda x:\
                                                     datetime.datetime.strptime(x.CurrentStatusDate, '%m/%d/%Y'), axis=1)


## Pluto

In [ ]:
pluto13_m = gpd.read_file('data/pluto13_m.geojson')
pluto14_m = gpd.read_file('data/pluto14_m.geojson')
pluto15_m = gpd.read_file('data/pluto15_m.geojson')

In [ ]:
vol_ct13 = vol13[vol13.BoroID==1].groupby('BBL_new').count()['Unnamed: 0']
pluto13_m.set_index('BBL', inplace=True)
pluto13_m['vol_ct'] = vol_ct13
pluto13_m['vol_ct'][pluto13_m.vol_ct.isna()] = 0

vol_ct14 = vol14[vol14.BoroID==1].groupby('BBL_new').count()['Unnamed: 0']
pluto14_m.set_index('BBL', inplace=True)
pluto14_m['vol_ct'] = vol_ct14
pluto14_m['vol_ct'][pluto14_m.vol_ct.isna()] = 0

vol_ct15 = vol15[vol15.BoroID==1].groupby('BBL_new').count()['Unnamed: 0']
pluto15_m.set_index('BBL', inplace=True)
pluto15_m['vol_ct'] = vol_ct15
pluto15_m['vol_ct'][pluto15_m.vol_ct.isna()] = 0
